In [116]:
import blurbs

import numpy as np
import matplotlib.pyplot as plt
import csv
from functools import reduce
import re 
from collections import defaultdict

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import pandas as pd

import math



# imports from my modules
import imp
import blurbs
imp.reload(blurbs)
import feature_generation
imp.reload(feature_generation)
import bullets
imp.reload(bullets)


# yay neural nets
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.activations import linear, relu, sigmoid


In [152]:
# We need to add 'zip' column to addr_df by merging with apt_df.
DIR = '/home/bryce/Projects/Data_Science/Apt_Prices/csvs/march8/'

address_file =  DIR + 'pd_address_info.csv' 
apt_file = DIR + 'pd_apt_info.csv'

addr_df = pd.read_csv(address_file, sep=';')
apt_df = pd.read_csv(apt_file, sep=';')


addrs_and_zips = apt_df[['address', 'zip']].drop_duplicates()
addr_df = addr_df.merge(addrs_and_zips, on='address')
apt_df.drop(columns=['zip'], inplace=True)
# The number of addresses might go down because we've thrown out apartments with missing data (say, no price)


In [153]:
((apt_train, addr_train), (apt_test, addr_test)) = feature_generation.split_data(apt_df=apt_df, addr_df=addr_df)

print("addr_train shape: ", addr_train.shape)
print("addr_test shape: ", addr_test.shape)




apt_df shape before:  (3166, 7)
addr_df shape before:  (400, 8)
apt_df shape after:  (3151, 7)
addr_df shape after:  (292, 8)
addr_train shape:  (204, 8)
addr_test shape:  (88, 8)


/home/bryce/Projects/Data_Science/Apt_Prices/feature_generation.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addr_df.drop_duplicates(subset=['address'], inplace=True)


In [154]:
feat_gen = feature_generation.FeatureGenerator(num_blurb_components = 5, num_bullet_components = 10)
(X_train, y_train) = feat_gen.get_training_features(apt_df_train=apt_train, addr_df_train=addr_train)
(X_test, y_test) = feat_gen.get_testing_features(apt_df_test=apt_test, addr_df_test=addr_test)

training svd matrix shape  (204, 5)
about to return the training_svd_df, with shape (204, 5)
columns before adding bullet feats:  ['address', 'url', 'title', 'bullets', 'blurb', 'latlng', 'latlng_len', 'zip', 'lat', 'lng', 'lat^2', 'lng^2', 'lat * lng', 'Blurb_svd_1', 'Blurb_svd_2', 'Blurb_svd_3', 'Blurb_svd_4', 'Blurb_svd_5']
Bryce?
here are the columns again in _create_combined_bullets:  ['address', 'url', 'title', 'bullets', 'blurb', 'latlng', 'latlng_len', 'zip', 'lat', 'lng', 'lat^2', 'lng^2', 'lat * lng', 'Blurb_svd_1', 'Blurb_svd_2', 'Blurb_svd_3', 'Blurb_svd_4', 'Blurb_svd_5']
Richards?
Bryce before hand_picked_features, addr_feats.columns ['address', 'url', 'title', 'bullets', 'blurb', 'latlng', 'latlng_len', 'zip', 'lat', 'lng', 'lat^2', 'lng^2', 'lat * lng', 'Blurb_svd_1', 'Blurb_svd_2', 'Blurb_svd_3', 'Blurb_svd_4', 'Blurb_svd_5', 'combined_bullets']
Bryce?
here are the columns again in _create_combined_bullets:  ['address', 'url', 'title', 'bullets', 'blurb', 'latlng', 'la

Steps before we train the neural net

1) generate train/val/test data and inspect for sanity (DONE)
2) generate training and validation features, and inspect for sanity

In [120]:
all_col_names = list(X_train.columns)
print(all_col_names)

['address', 'beds', 'baths', 'sq_ft', 'location', 'units_in_building', 'url', 'title', 'bullets', 'blurb', 'latlng', 'latlng_len', 'zip', 'lat', 'lng', 'lat^2', 'lng^2', 'lat * lng', 'Blurb_svd_1', 'Blurb_svd_2', 'Blurb_svd_3', 'Blurb_svd_4', 'Blurb_svd_5', 'combined_bullets', 'Bullet_concept_0', 'Bullet_concept_1', 'Bullet_concept_2', 'Bullet_concept_3', 'Bullet_concept_4', 'Bullet_concept_5', 'Bullet_concept_6', 'Bullet_concept_7', 'Bullet_concept_8', 'Bullet_concept_9', 'bullets_feat_has_security', 'bullets_feat_has_lawn', 'bullets_feat_are_pets_allowed', 'bullets_feat_are_no_pets_allowed', 'bullets_feat_has_pool', 'bullets_feat_is_waterfront', 'bullets_feat_is_furnished', 'bullets_feat_has_elevator', 'bullets_feat_kitchen_quality_score', 'bullets_feat_upscale_score', 'bullets_feat_recreation_score', 'bullets_feat_luxury_score', 'furnished', 'zip_29401', 'zip_29403', 'zip_29405', 'zip_29406', 'zip_29407', 'zip_29412', 'zip_29414', 'zip_29464', 'zip_29492', 'property_type', 'property

In [207]:
all_col_names = X_train.columns
cols = feature_generation.select_column_names(all_column_names=all_col_names,
                                              latlng = False,
                                              units_in_building = True,
                                              zip_feats = True,
                                              bullets_svd_feats = True,
                                              bullets_handpicked_feats=True,
                                              property_type = True,
                                              furnished=True,
                                              blurb_feats = False,
                                              cross_term_res=[])
print(cols)
X_train_feats = X_train[cols]

normalizer = Normalization(axis=-1)
normalizer.adapt(X_train_feats)

['beds', 'baths', 'sq_ft', 'units_in_building', 'zip_29401', 'zip_29403', 'zip_29406', 'zip_29407', 'zip_29412', 'zip_29414', 'zip_29464', 'zip_29492', 'Bullet_concept_0', 'Bullet_concept_1', 'Bullet_concept_2', 'Bullet_concept_3', 'Bullet_concept_4', 'Bullet_concept_5', 'Bullet_concept_6', 'Bullet_concept_7', 'Bullet_concept_8', 'Bullet_concept_9', 'bullets_feat_has_security', 'bullets_feat_has_lawn', 'bullets_feat_are_pets_allowed', 'bullets_feat_are_no_pets_allowed', 'bullets_feat_has_pool', 'bullets_feat_is_waterfront', 'bullets_feat_is_furnished', 'bullets_feat_has_elevator', 'bullets_feat_kitchen_quality_score', 'bullets_feat_upscale_score', 'bullets_feat_recreation_score', 'bullets_feat_luxury_score', 'property_type_house', 'property_type_townhouse', 'property_type_undefined', 'furnished']


In [212]:


model = tf.keras.models.Sequential([
    normalizer,
    Dense(20, activation='relu', name='L2', kernel_regularizer=tf.keras.regularizers.L2(0.2)),
    Dense(1, activation='linear', name='output')
], name = 'FirstModel')

model.compile(loss='mean_absolute_error',
              optimizer=tf.keras.optimizers.Adam(0.005))




In [ ]:
# Pick params systematically
def search_hyperparams(
    X_train, y_train, X_test, y_test, epochs = 1500,
    node_layer_architectures = [(5), (10), (20), (40), (10, 5), (20, 10), (40, 20), (10, 10, 10), (20, 10, 5)],
    lambdas = [0.01, .05, 0.1, 0.2, 0.5, 1.0]):
    model_name_to_model = dict()
    model_name_to_history = dict()
    model_num = 0
    best_testing_result = 9999999
    best_model = ""
    for lambda_ in lambdas:
        for node_sequence in node_layer_architectures:
            # Here we'll construct the layers of our neural net
            layers = []
            # Add all the hidden layers
            for i in range(len(node_sequence)):
                n_nodes = node_sequence[i]
                name = "Layer_" + str(i)
                layer = Dense(n_nodes, activation='relu', name=name, kernel_regularizer=tf.keras.regularizers.L2(lambda_))
                layers.append(layer)
            # Add the output layer
            layers.append(Dense(1, activation='linear'))
            model_name = "hidden-layers( {nodes} )_lambda( {lambda_} )".format(
                nodes=str(node_sequence), lambda_=str(lambda_))
            model = model = tf.keras.models.Sequential(
                normalizer=normalizer,
                layers=layers, 
                name=model_name)
            print("About to train model: ", model_name)
            history = model.fit(X_train, y_train, epochs=epochs, verbose=0)
            model_name_to_history[model_name] = history 
            model_name_to_model[model_name] = model
            results_train = model.evaluate(X_train, y_train, verbose=0)
            results_test = model.evaluate(X_test, y_test, verbose=0)
            print("Model ", model_name, "'s performance:")
            print("Training: ", results_train, ". Testing: ", results_test)
            if results_test < best_testing_result:
                best_testing_result = results_test
                best_model = model_name
    print("The best model was " + best_model + " with testing result: " + best_testing_result)
    return (model_name_to_model, model_name_to_history)
            


In [209]:
import xgboost as xgb

xgbmodel = xgb.XGBRegressor()
xgbmodel.fit(X_train_feats, y_train)
yhat = xgbmodel.predict(X_train_feats)

In [210]:
yhat = np.reshape(yhat, (yhat.shape[0], 1))
diff = yhat - y_train
np.average(np.sqrt(np.square(diff, diff)))

46.72101754257359

In [215]:
history = model.fit(X_train_feats, y_train, epochs=2000, verbose=1)


Epoch 1/2000
49/49 [==============================] - 0s 1ms/step - loss: 158.5122
Epoch 2/2000
49/49 [==============================] - 0s 1ms/step - loss: 157.9274
Epoch 3/2000
49/49 [==============================] - 0s 1ms/step - loss: 158.9793
Epoch 4/2000
49/49 [==============================] - 0s 1ms/step - loss: 157.9338
Epoch 5/2000
49/49 [==============================] - 0s 1ms/step - loss: 159.3910
Epoch 6/2000
49/49 [==============================] - 0s 1ms/step - loss: 160.5567
Epoch 7/2000
49/49 [==============================] - 0s 1ms/step - loss: 158.4532
Epoch 8/2000
49/49 [==============================] - 0s 923us/step - loss: 158.0431
Epoch 9/2000
49/49 [==============================] - 0s 933us/step - loss: 158.6767
Epoch 10/2000
49/49 [==============================] - 0s 895us/step - loss: 157.3784
Epoch 11/2000
49/49 [==============================] - 0s 978us/step - loss: 158.1471
Epoch 12/2000
49/49 [==============================] - 0s 968us/step - loss: 

KeyboardInterrupt: 

In [216]:
results_train = model.evaluate(X_train_feats, y_train)
results_test = model.evaluate(X_test[cols], y_test)
print("training results: ", results_train)
print("test results: ", results_test)



33/33 [==============================] - 0s 861us/step - loss: 541.4651
training results:  155.9171600341797
test results:  541.4651489257812


In [139]:
yhat_test = model.predict(X_test[cols])
y_test_diff = y_test - yhat_test
yhat_train = model.predict(X_train[cols])
y_train_diff = y_train - yhat_train

61/61 [==============================] - 0s 674us/step


In [117]:
print(X_test.shape)
print(X_train.shape)

(827, 112)
(1777, 112)


In [140]:
from bullets import BulletFeatures
import copy 
def create_results_df_for_analysis(X, y, yhat):
    D = copy.copy(X)
    D['yhat'] = yhat
    D['y'] = y
    D['diff'] = D.yhat - D.y
    # Now let's drop the dummy vars and cross terms
    columns_to_drop = D.filter(regex='.*zip_.*|.*svd.*|.* x .*|property_type_.*')
    print(list(columns_to_drop))
    D = D.drop(columns_to_drop, axis=1)
    return D
    


In [141]:
D_test = create_results_df_for_analysis(X_test, y=y_test, yhat=yhat_test)
D_train = create_results_df_for_analysis(X_train, y=y_train, yhat=yhat_train)

['svd_1', 'svd_2', 'svd_3', 'svd_4', 'svd_5', 'svd_6', 'svd_7', 'svd_8', 'svd_9', 'svd_10', 'svd_11', 'svd_12', 'svd_13', 'svd_14', 'svd_15', 'svd_16', 'svd_17', 'svd_18', 'svd_19', 'svd_20', 'svd_21', 'svd_22', 'svd_23', 'svd_24', 'svd_25', 'svd_26', 'svd_27', 'svd_28', 'svd_29', 'svd_30', 'zip_29401', 'zip_29403', 'zip_29406', 'zip_29407', 'zip_29412', 'zip_29414', 'zip_29464', 'zip_29485', 'zip_29492', 'property_type_condo', 'property_type_house', 'property_type_townhouse', 'property_type_undefined', 'baths x baths', 'baths x zip_29401', 'baths x zip_29403', 'baths x zip_29406', 'baths x zip_29407', 'baths x zip_29412', 'baths x zip_29414', 'baths x zip_29464', 'baths x zip_29485', 'baths x zip_29492', 'beds x beds', 'beds x zip_29401', 'beds x zip_29403', 'beds x zip_29406', 'beds x zip_29407', 'beds x zip_29412', 'beds x zip_29414', 'beds x zip_29464', 'beds x zip_29485', 'beds x zip_29492', 'sq_ft x sq_ft', 'sq_ft x zip_29401', 'sq_ft x zip_29403', 'sq_ft x zip_29406', 'sq_ft x z

In [145]:
print(X_test.furnished.value_counts())

0    647
1     16
Name: furnished, dtype: int64


In [146]:
D_test.to_csv('/home/bryce/Projects/Data_Science/Apt_Prices/csvs/march8/test_nn_results.csv', sep=';', index=False)
D_train.to_csv('/home/bryce/Projects/Data_Science/Apt_Prices/csvs/march8/train_nn_results.csv', sep=';', index=False)

In [75]:
X_test['yhat'] = yhat_test
X_test['y'] = y_test
X_test['diff'] = X_test.yhat - X_test.y

In [80]:
from bullets import BulletFeatures
bullet_feats = BulletFeatures()
bullets_df = bullet_feats._create_combined_bullets(X_test)
R_test = X_test.merge(bullets_df[['address', 'combined_bullets']], on='address')

We made it here!
But did we make it here?


In [81]:
columns_to_save = feature_generation.select_column_names(all_column_names=all_col_names,
                                              latlng = False,
                                              units_in_building = True,
                                              zip_feats = False,
                                              bullet_feats = True,
                                              property_type = False,
                                              furnished=True,
                                              blurb_feats = False,
                                              cross_term_res=[])
columns_to_save.extend(['combined_bullets', 'zip', 'yhat', 'y', 'diff', 'property_type'])
print(columns_to_save)
print(list(R_test.columns))

['beds', 'baths', 'sq_ft', 'units_in_building', 'Bullet_concept_0', 'Bullet_concept_1', 'Bullet_concept_2', 'Bullet_concept_3', 'Bullet_concept_4', 'Bullet_concept_5', 'Bullet_concept_6', 'Bullet_concept_7', 'Bullet_concept_8', 'Bullet_concept_9', 'furnished', 'zip', 'yhat', 'y', 'diff', 'property_type']
['address', 'beds', 'baths', 'sq_ft', 'location', 'units_in_building', 'url', 'title', 'bullets', 'blurb', 'latlng', 'latlng_len', 'zip', 'lat', 'lng', 'lat^2', 'lng^2', 'lat * lng', 'svd_1', 'svd_2', 'svd_3', 'svd_4', 'svd_5', 'svd_6', 'svd_7', 'svd_8', 'svd_9', 'svd_10', 'svd_11', 'svd_12', 'svd_13', 'svd_14', 'svd_15', 'svd_16', 'svd_17', 'svd_18', 'svd_19', 'svd_20', 'svd_21', 'svd_22', 'svd_23', 'svd_24', 'svd_25', 'svd_26', 'svd_27', 'svd_28', 'svd_29', 'svd_30', 'Bullet_concept_0', 'Bullet_concept_1', 'Bullet_concept_2', 'Bullet_concept_3', 'Bullet_concept_4', 'Bullet_concept_5', 'Bullet_concept_6', 'Bullet_concept_7', 'Bullet_concept_8', 'Bullet_concept_9', 'furnished', 'zip_29

In [62]:
def get_diff_df():
    D = X_test
    D = D.merge(addr_test, on='address')
    D['y'] = y_test
    D['yhat'] = yhat 
    D['diff'] = D.y - D.yhat 
    D.sort_values(by='diff', inplace=True)
    D.to_csv('/tmp/D.csv', sep=';')
    return D

import time

def scroll_through_overestimates():
    D = get_diff_df()
    i = 0
    while True:
            row = D.iloc[i]
            print(row.address, row.url, "beds and baths: ", row.beds, row.baths, "price: ", row.y, "estimated: ", row.yhat)
            i += 1
            time.sleep(10)


def scroll_through_underestimates():
    D = get_diff_df()
    i = D.shape[0] - 1
    while True:
            row = D.iloc[i]
            print(row.address, row.url, "beds and baths: ", row.beds, row.baths, "price: ", row.y, "estimated: ", row.yhat)
            i -= 1
            time.sleep(10)


In [63]:
scroll_through_underestimates()

13 Logan St, Charleston, SC 29401  https://www.apartments.com/4-br-4-bath-house-13-logan-street-charleston-sc/rqbyr41/ beds and baths:  4 4.0 price:  20000 estimated:  6697.7485
15 West St, Charleston, SC 29401  https://www.apartments.com/15-west-st-charleston-sc/lcjrtpf/ beds and baths:  2 2.25 price:  6000 estimated:  3117.2612
15 West St, Charleston, SC 29401  https://www.apartments.com/15-west-st-charleston-sc/lcjrtpf/ beds and baths:  2 2.25 price:  6000 estimated:  3117.2612
1123 Blakeway St, Charleston, SC 29492  https://www.apartments.com/1123-blakeway-st-daniel-island-sc/wqdexbl/ beds and baths:  3 2.0 price:  5700 estimated:  3570.7751
21 George St, Charleston, SC 29401  https://www.apartments.com/21-george-st-charleston-sc-unit-410/lmxl6nd/ beds and baths:  2 2.0 price:  5250 estimated:  3135.413
21 George St, Charleston, SC 29401  https://www.apartments.com/21-george-st-charleston-sc-unit-410/lmxl6nd/ beds and baths:  2 2.0 price:  4500 estimated:  3037.1042
1 Percy St, Cha

KeyboardInterrupt: 